# Working Through the Left Ventricle Segmentation Tutorial

This is an attempt to work through the [Fourier based tutorial](https://www.kaggle.com/c/second-annual-data-science-bowl/details/fourier-based-tutorial) that uses image segmentation and Fourier analysis to select the most important parts of the heart MRI images.

- Link to the Left Ventricle Segmentation Tutorial as a [GitHub Gist](https://gist.github.com/ajsander/fb2350535c737443c4e0#file-tutorial-md) or the [post on the Kaggle website](https://www.kaggle.com/c/second-annual-data-science-bowl/details/fourier-based-tutorial).


- All of the Python code for the tutorial is in the file [segment.py](https://raw.githubusercontent.com/booz-allen-hamilton/DSB2/master/segment.py) on the [booz-allen-hamilton/DSB2 github website](https://github.com/booz-allen-hamilton/DSB2)


- The method in the tutorial is based on the article "Automated Detection of Left Ventricle in 4D MR Images: Experience from a Large Study" by [Lin, et al. (2006)](https://www.researchgate.net/publication/6452142_Automated_Detection_of_Left_Ventricle_in_4D_MR_Images_Experience_from_a_Large_Study)


## Software to install

To get the tutorial to run, I needed to install the following software on Windows 7 Professional.  The links below should provide good instructions.  **Please see me if you need help.**

- [Jupyter and Anaconda Python](http://jupyter.readthedocs.org/en/latest/install.html#new-to-python-and-jupyter)


- The tutorial I used to get OpenCV to work on Windows 7 is [OpenCV (Open Computer Vision) installation on Windows for use as a Python package](http://mathalope.co.uk/2015/05/07/opencv-python-how-to-install-opencv-python-package-to-anaconda-windows/).  If you know how to create and modify environment variables, I encourage you to complete the section on installing FFMPEG.  If not, you **might not need to install the FFMPEG (video codec)** since we are processing images (not videos).  Note: There are [other ways to install OpenCV given on StackOverflow](http://stackoverflow.com/questions/23119413/how-to-install-python-opencv-through-conda)


- The pydicom package for python to read dicom image files can be installed running the following command at the windows command prompt.

    `pip install pydicom`
    

- [Mango desktop dicom file viewer for Windows](http://ric.uthscsa.edu/mango/mango.html)

## Abbreviations and definitions of terms used in the tutorial

- LV = Left Ventricle

- Image segmentation: "The goal of image segmentation is to divide an image into a set of semantically meaningful, homogeneous, and nonoverlapping regions of similar attributes such as intensity, depth, color, or texture. The segmentation result is either an image of labels identifying each homogeneous region or a set of contours which describe the region boundaries." [MRI Segmentation of the Human Brain: Challenges, Methods, and Applications](http://www.hindawi.com/journals/cmmm/2015/450341/)

- ROI = Region Of Interest in the image, usually circular

- Short axis stack: The [University of Minnesota Atlas of Cardiac Anatomy](http://www.vhlab.umn.edu/atlas/cardiac-mri/index.shtml) has a [brief description of the short axis stack along with a drop down menu allowing you to choose different hearts to view](http://www.vhlab.umn.edu/atlas/cardiac-mri/short-axis-stack/index.shtml).  They say "A short axis stack shows the complete series of images taken during an MRI stack sequence scan. These images are taken at 1-4mm intervals in a plane perpendicular to the long axis of the left ventricle."


## Load packages into python

In [1]:
#
# segment.py
#
import cv2
import numpy as np
import dicom
import json
import os
import shutil
import sys
import random
from matplotlib import image
from scipy.ndimage import label
from scipy.ndimage.morphology import binary_erosion
from scipy.fftpack import fftn, ifftn
from scipy.signal import argrelmin, correlate
from scipy.spatial.distance import euclidean
from scipy.stats import linregress
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline

## Set some parameters

In [2]:
# number of bins to use in histogram for gaussian regression
NUM_BINS = 100

# number of standard deviations past which we will consider a pixel an outlier
STD_MULTIPLIER = 2

# number of points of our interpolated dataset to consider when searching for
# a threshold value; the function by default is interpolated over 1000 points,
# so 250 will look at the half of the points that is centered around the known
# myocardium pixel
THRESHOLD_AREA = 250

# number of pixels on the line within which to search for a connected component
# in a thresholded image, increase this to look for components further away
COMPONENT_INDEX_TOLERANCE = 20

# number of angles to search when looking for the correct orientation
ANGLE_SLICES = 36

## Construct a status log function

In [3]:
# status log
def log(msg, lvl):
    string = ""
    for i in range(lvl):
        string += " "
    string += msg
    print string

## Load data